In [119]:
import pandas as pd
import numpy as np 
from itertools import permutations
import math
import random

In [135]:
#generate a tree to obtain a additive distance matrix 

class Vertex:
    def __init__(self, node):
        self.id = node
        self.adjacent = {}

    def __str__(self):
        return str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    def get_connections(self):
        return self.adjacent.keys()  

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent[neighbor]

class MatrixTree:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, cost = 0):
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

    def get_vertices(self):
        return self.vert_dict.keys()

In [136]:
def get_tree():   
    output = (np.array([random.randrange(1, 10) for _ in range(0, 5)]))

    #generate n+1 random edge weights where n = # of nodes + a weight for a connecting branch

    t = MatrixTree()

    t.add_vertex('a')
    t.add_vertex('b')
    t.add_vertex('c')
    t.add_vertex('d')

    t.add_edge('a', 'b', output[0] + output[1])   #dist between a to b
    t.add_edge('a', 'c', output[0] + output[4] + output[2]) #distance between a, connecting limb, c
    t.add_edge('a', 'd', output[0] + output[4] + output[3]) 
    t.add_edge('b', 'c', output[1] + output[4] + output[2])
    t.add_edge('b', 'd', output[1] + output[4] + output[3])
    t.add_edge('c', 'd', output[2] + output[3])
    return t

In [137]:
# get dict of dicts for each node, neighbors, distance to neighbors 
def get_matrix(tree):
    D = {}
    for v in tree:
        vid = v.get_id()
        D[vid] = {vid : 0} # assign distance from itself to 0
        for w in v.get_connections():
            wid = w.get_id()
            D[vid][wid] = int(v.get_weight(w))
    return D


In [138]:
mat= pd.DataFrame.from_dict(get_matrix(t)) #convert to 2d matrix
#visualize as n x n distance matrix         
mat

,a,b,c,d
a,0,7,15,16
b,7,0,18,19
c,15,18,0,15
d,16,19,15,0


In [124]:
#check if the matrix is additive using four point condition
def is_additive(matrix, labels):
    i = labels[0]
    j = labels[1] 
    k = labels[2] 
    l = labels[3] 
    result = ''
    if ((matrix[i][j] + matrix[k][l] <= matrix[i][k] + matrix[j][l]) and 
    (matrix[i][k] + matrix[j][l] == matrix[i][l] + matrix[j][k])):
        result = "Additive"
    else:
        result = "Not additive"
    return result 

labels = ['a','b','c','d']
is_additive(mat, labels)

'Additive'

In [30]:
#dm = pd.DataFrame.from_dict(tree)
dm['A']['B'] = 8
dm['A']['C'] = 15
dm['A']['D'] = 9
dm['B']['C'] = 5
dm['B']['D'] = 10
dm['C']['D'] = 11

dm['B']['A'] = 8
dm['C']['A'] = 15
dm['D']['A'] = 9
dm['C']['B'] = 5
dm['D']['B'] = 10
dm['D']['C'] = 11
dm

,A,B,C,D
A,0,8,15,9
B,8,0,5,10
C,15,5,0,11
D,9,10,11,0


## Neigbor join

In [64]:
from ete3 import Tree
import copy
clusters = []
for cluster in list(dm):
    clusters.append(cluster)
cluster2idx = { name : idx for idx, name in enumerate(clusters) }
idx2cluster = copy.deepcopy(clusters)
m = copy.deepcopy(dm).to_dict()
Z = np.empty((0,4), float)
result = ""
print(clusters)
#membership = [set([cluster2idx[name]]) for name in clusters]

['A', 'B', 'C', 'D']


In [83]:
import copy
def NJ(dm):
    edge_list = []
    clusters = []
    for cluster in list(dm):
        clusters.append(cluster)
    cluster2idx = { name : idx for idx, name in enumerate(clusters) }
    idx2cluster = copy.deepcopy(clusters)
    m = copy.deepcopy(dm).to_dict()
    Z = np.empty((0,4), float)
    result = ""
    #print(clusters)
    #membership = [set([cluster2idx[name]]) for name in clusters]
    M, N = dm.shape
    new_cluster_idx = M+1 
    #print(T)
    while len(clusters) > 1:
        #print(result)
        #print(m)
        idx1 = None
        idx2 = None
        ui = 0
        uj = 0
        Sd = 100000
        for i in clusters:
            for j in clusters:
                if i != j:
                    ui_t = sum(m[i][k] for k in clusters)
                    uj_t = sum(m[j][k] for k in clusters)
                    curr = (M - 2) * m[i][j] - ui_t - uj_t
                    if Sd > curr:
                        Sd = curr
                        idx1 = cluster2idx[i]
                        idx2 = cluster2idx[j]
                        ui = ui_t
                        uj = uj_t
        #print("ui", ui)
        #print("uj", uj)
        #print(Sd)

        Dij = m[idx2cluster[idx1]][idx2cluster[idx2]]
        new_cluster = "(" + idx2cluster[idx1] +"." + idx2cluster[idx2] + ")"
        result = new_cluster
        m[new_cluster] = {}
        new_cluster_idx += 1
        #membership.append(membership[cluster2idx[idx1]] | membership[cluster2idx[idx2]])
        #Z = np.append(Z, np.array([[idx1, idx2, Sd]]))
        if M - 2 != 0:            
            m[idx2cluster[idx1]][new_cluster] = (float(m[idx2cluster[idx1]][idx2cluster[idx2]]) + (1/(M-2)) * (ui - uj))/2
            m[idx2cluster[idx2]][new_cluster] = (float(m[idx2cluster[idx1]][idx2cluster[idx2]]) + (1/(M-2)) * (uj - ui))/2
            edge_list.append((new_cluster, idx2cluster[idx1], m[idx2cluster[idx1]][new_cluster]))
            edge_list.append((new_cluster, idx2cluster[idx2], m[idx2cluster[idx2]][new_cluster]))
        else:
            edge_list.append((idx2cluster[idx1], idx2cluster[idx2], m[idx2cluster[idx1]][idx2cluster[idx2]]))
#             edge_list.append((root, idx2cluster[idx1], m[idx2cluster[idx1]][idx2cluster[idx2]]/2))
#             edge_list.append((root, idx2cluster[idx2], m[idx2cluster[idx1]][idx2cluster[idx2]]/2))
            #T[idx2cluster[idx2]] = {idx2cluster[idx2]:m[idx2cluster[idx2]][new_cluster]}
        #if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
        #if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]

        clusters.remove(idx2cluster[idx1])
        clusters.remove(idx2cluster[idx2])

        m[new_cluster][new_cluster] = 0
        for c in clusters:
            m[c][new_cluster] = (m[c][idx2cluster[idx1]] + m[c][idx2cluster[idx2]] - Dij)/2
            m[new_cluster][c] = m[c][new_cluster]
#             try:
#                 T[c][new_cluster] = {new_cluster:(m[c][idx2cluster[idx1]] + m[c][idx2cluster[idx2]] - Dij)/2}
#             except:
#                 T[c] = {}
#                 T[c][new_cluster] = {idx2cluster:(m[c][idx2cluster[idx1]] + m[c][idx2cluster[idx2]] - Dij)/2}
            for k in list(m[c].keys()):
                if k == idx2cluster[idx1] or k == idx2cluster[idx2]:
                    del m[c][k]

        del cluster2idx[idx2cluster[idx1]]
        del cluster2idx[idx2cluster[idx2]]
        if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
        if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]

        clusters.append(new_cluster)
        cluster2idx[new_cluster] = len(idx2cluster)
        idx2cluster.append(new_cluster)
        #print("current tree", T)
        #print("current matrix", m)
        M -= 1
        N -= 1
    return T, result, edge_list

In [84]:
import ete3
tree = {'A': {'A': 0, 'B': 6, 'C': 6, 'D': 8},
 'B': {'A': 6, 'B': 0, 'C': 8, 'D': 2},
 'C': {'A': 6, 'B': 8, 'C': 0, 'D': 10},
 'D': {'A': 8, 'B': 2, 'C': 10, 'D': 0}}
dm1 = pd.DataFrame.from_dict(tree)
dm1['A']['B'] = 4
dm1['B']['A'] = 4
dm1['A']['C'] = 10
dm1['C']['A'] = 10
dm1['A']['D'] = 9
dm1['D']['A'] = 9
dm1['B']['C'] = 8
dm1['C']['B'] = 8
dm1['B']['D'] = 7
dm1['D']['B'] = 7
dm1['C']['D'] = 9
dm1['D']['C'] = 9
nj_tree1, nj_result1, nj_list1 = NJ(dm1)
ete_nj_tree = Tree.from_parent_child_table(nj_list1)
print(ete_nj_tree)
print(ete_nj_tree.write())


   /-A
  |
--|--B
  |
  |   /-C
   \-|
      \-D
(A:3,B:1,(C:5,D:4)1:2);


In [58]:
##additive phlyogeny construction using degenerate triples

In [59]:

tree = {'A': {'A': 0, 'B': 6, 'C': 6, 'D': 8},
 'B': {'A': 6, 'B': 0, 'C': 8, 'D': 2},
 'C': {'A': 6, 'B': 8, 'C': 0, 'D': 10},
 'D': {'A': 8, 'B': 2, 'C': 10, 'D': 0}}
dm1 = pd.DataFrame.from_dict(tree)
dm1['A']['B'] = 4
dm1['B']['A'] = 4
dm1['A']['C'] = 10
dm1['C']['A'] = 10
dm1['A']['D'] = 9
dm1['D']['A'] = 9
dm1['B']['C'] = 8
dm1['C']['B'] = 8
dm1['B']['D'] = 7
dm1['D']['B'] = 7
dm1['C']['D'] = 9
dm1['D']['C'] = 9

dm['A']['B'] = 8
dm['A']['C'] = 15
dm['A']['D'] = 9
dm['B']['C'] = 5
dm['B']['D'] = 10
dm['C']['D'] = 11

dm['B']['A'] = 8
dm['C']['A'] = 15
dm['D']['A'] = 9
dm['C']['B'] = 5
dm['D']['B'] = 10
dm['D']['C'] = 11

def check_and_find_degenerate(D):
    cols = 'ABCD'
    triplets = permutations(D.index,3)
    for triplet in triplets:
        i,j,k = triplet
        if D[i][j] + D[j][k] == D[i][k]:
            return i,j,k, True
        
    return -1,-1,-1, False
    
def compute_trimming_param(D):
    cols = 'ABCD'
    triplets = permutations(D.index,3)
    min_delta = float('inf')
    best_i,best_j,best_k = -1,-1,-1
    for triplet in triplets:
        i,j,k = triplet
        delta = (D[i][j] + D[j][k] - D[i][k])/2
        if delta > 0 and delta < min_delta:
            min_delta = delta
            best_i,best_j,best_k = i,j,k
            
    return min_delta, best_i,best_j,best_k

def Trim(D, delta):
    D = D - (2*delta)
    for i in D.index:
        D[i][i] = 0
    return D

def additive_phylogeny(D, v):
    if D.shape == (2,2):
        T = {D.index[0] : {D.index[1]: D[D.index[0]][D.index[1]]}, 
             D.index[1] : {D.index[0]: D[D.index[1]][D.index[0]]}}
        return T
    i,j,k,flag = check_and_find_degenerate(D)
    if not flag:
        delta,best_i,best_j,best_k = compute_trimming_param(D)
        i,j,k = best_i,best_j,best_k
        D = Trim(D, delta)
    x = D[i][j]
    D = D.drop([j])
    D = D.drop(j, axis=1)
    T = additive_phylogeny(D, v+1)
    T[v] = {i:x}
    T[i] = {v:x}
    if v + 1 in T:
        del T[v+1][i]
        T[v][v+1] = D[i][k]-x-T[v+1][k]
        T[v+1][v] = D[i][k]-x-T[v+1][k]
    else:
        T[v][k] = D[i][k]-x
        T[k] = {v:D[i][k]-x}
    
    T[v][j] = 0
    T[j] = {v:0}
    for l in T:
        if l in D.index and j in D.index and T[l][v] != D[l][j]:
            print('Matrix not additive')
            return
    for v in T:
        for l in 'ABCD':
            if l in T[v] and type(v) == int:
                T[l][v] =  T[l][v] + (delta)
                T[v][l] =  T[v][l] + (delta)
            
    return T


In [131]:
from ete3 import Tree
T = additive_phylogeny(dm1, 0)
print(T)
edge_list = []
for v1 in T:
    for v2 in T[v1]:
        if type(v1) == int and (v1,v2,T[v1][v2]) not in edge_list and (v2,v1,T[v2][v1]) not in edge_list:
            edge_list.append((v1,v2, T[v1][v2]))
print(edge_list)
tree = Tree.from_parent_child_table(edge_list)
print(tree.write())
ete_nj_tree.robinson_foulds(tree,unrooted_trees=True)[0]
print(ete_nj_tree)
print(tree)

{'A': {0: 3.0}, 'C': {1: 5.0}, 1: {'C': 5.0, 'D': 4.0, 0: 2.0}, 'D': {1: 4.0}, 0: {'A': 3.0, 1: 2.0, 'B': 1.0}, 'B': {0: 1.0}}
[(1, 'C', 5.0), (1, 'D', 4.0), (1, 0, 2.0), (0, 'A', 3.0), (0, 'B', 1.0)]
(C:5,D:4,(A:3,B:1)1:2);

   /-A
  |
--|--B
  |
  |   /-C
   \-|
      \-D

   /-C
  |
--|--D
  |
  |   /-A
   \-|
      \-B


In [132]:
def create_edge_list(T):
    edge_list = []
    for v1 in T:
        for v2 in T[v1]:
            if type(v1) == int and (v1,v2,T[v1][v2]) not in edge_list and (v2,v1,T[v2][v1]) not in edge_list:
                edge_list.append((v1,v2, T[v1][v2]))
    return edge_list

In [143]:
mat= pd.DataFrame.from_dict(get_matrix(get_tree())) #convert to 2d matrix
#visualize as n x n distance matrix         
mat
T = additive_phylogeny(dm1, 0)
ete_dt_tree = Tree.from_parent_child_table(create_edge_list(T))
nj_tree, nj_result, nj_list = NJ(dm1)
ete_nj_tree = Tree.from_parent_child_table(nj_list)
ete_nj_tree.robinson_foulds(ete_dt_tree,unrooted_trees=True)[0]

0

In [149]:
T = additive_phylogeny(mat, 0)
print(mat)
ete_dt_tree = Tree.from_parent_child_table(create_edge_list(T))
print(ete_dt_tree.write())
nj_tree, nj_result, nj_list = NJ(mat)
ete_nj_tree = Tree.from_parent_child_table(nj_list)
print(ete_nj_tree.write())
ete_nj_tree.robinson_foulds(ete_dt_tree,unrooted_trees=True)[0]

    a   b   c   d
a   0  12  16  16
b  12   0  18  18
c  16  18   0   4
d  16  18   4   0
(b:2,d:6,(a:12,c:0)1:-6);
(a:5,b:7,(c:2,d:2)1:9);


2

In [150]:
labels = ['a','b','c','d']
is_additive(mat, labels)

'Additive'

In [158]:
cols = 'abcd'
labels = permutations(cols,4)

In [159]:
check = []
for combs in labels:
    new_label = []
    for item in combs:
        new_label.append(item)
    print(new_label)
    check.append(is_additive(mat, new_label))
check

['a', 'b', 'c', 'd']
['a', 'b', 'd', 'c']
['a', 'c', 'b', 'd']
['a', 'c', 'd', 'b']
['a', 'd', 'b', 'c']
['a', 'd', 'c', 'b']
['b', 'a', 'c', 'd']
['b', 'a', 'd', 'c']
['b', 'c', 'a', 'd']
['b', 'c', 'd', 'a']
['b', 'd', 'a', 'c']
['b', 'd', 'c', 'a']
['c', 'a', 'b', 'd']
['c', 'a', 'd', 'b']
['c', 'b', 'a', 'd']
['c', 'b', 'd', 'a']
['c', 'd', 'a', 'b']
['c', 'd', 'b', 'a']
['d', 'a', 'b', 'c']
['d', 'a', 'c', 'b']
['d', 'b', 'a', 'c']
['d', 'b', 'c', 'a']
['d', 'c', 'a', 'b']
['d', 'c', 'b', 'a']


['Additive',
 'Additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Additive',
 'Additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Additive',
 'Additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Additive',
 'Additive']

In [163]:
import Bio
Bio.Phylo.Newick('(a:5,b:7,(c:2,d:2)1:9);')

TypeError: 'module' object is not callable

In [176]:
def check_matrix(newick_string):
    import io
    treedata = newick_string
    handle = io.StringIO(treedata)
    t = Phylo.read(handle, "newick")
    d = {}
    for x, y in itertools.combinations(t.get_terminals(), 2):
        v = t.distance(x, y)
        d[x.name] = d.get(x.name, {})
        d[x.name][y.name] = v
        d[y.name] = d.get(y.name, {})
        d[y.name][x.name] = v
    for x in t.get_terminals():
        d[x.name][x.name] = 0

    m = pd.DataFrame(d)
    return m

In [172]:
m

,a,b,c,d
a,0.0,12.0,16.0,16.0
b,12.0,0.0,18.0,18.0
c,16.0,18.0,0.0,4.0
d,16.0,18.0,4.0,0.0


In [173]:
tree = {'A': {'A': 0, 'B': 6, 'C': 6, 'D': 8},
 'B': {'A': 6, 'B': 0, 'C': 8, 'D': 2},
 'C': {'A': 6, 'B': 8, 'C': 0, 'D': 10},
 'D': {'A': 8, 'B': 2, 'C': 10, 'D': 0}}
dm1 = pd.DataFrame.from_dict(tree)
dm1['A']['B'] = 4
dm1['B']['A'] = 4
dm1['A']['C'] = 10
dm1['C']['A'] = 10
dm1['A']['D'] = 9
dm1['D']['A'] = 9
dm1['B']['C'] = 8
dm1['C']['B'] = 8
dm1['B']['D'] = 7
dm1['D']['B'] = 7
dm1['C']['D'] = 9
dm1['D']['C'] = 9

In [174]:
mat = dm1
T = additive_phylogeny(mat, 0)
print(mat)
ete_dt_tree = Tree.from_parent_child_table(create_edge_list(T))
print(ete_dt_tree.write())
nj_tree, nj_result, nj_list = NJ(mat)
ete_nj_tree = Tree.from_parent_child_table(nj_list)
print(ete_nj_tree.write())
ete_nj_tree.robinson_foulds(ete_dt_tree,unrooted_trees=True)[0]

    A  B   C  D
A   0  4  10  9
B   4  0   8  7
C  10  8   0  9
D   9  7   9  0
(C:5,D:4,(A:3,B:1)1:2);
(A:3,B:1,(C:5,D:4)1:2);


0

In [179]:
dm1 == check_matrix(ete_nj_tree.write())

,A,B,C,D
A,True,True,True,True
B,True,True,True,True
C,True,True,True,True
D,True,True,True,True
